In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile gpu_layers.h
#pragma once
#include <cuda_runtime.h>
#include <stdio.h>

#define CHECK_CUDA(call)                                                \
    do {                                                                \
        cudaError_t err = call;                                         \
        if (err != cudaSuccess) {                                       \
            fprintf(stderr, "CUDA Error %s:%d: %s\n",                   \
                    __FILE__, __LINE__, cudaGetErrorString(err));       \
            exit(EXIT_FAILURE);                                         \
        }                                                               \
    } while (0)

// NCHW layout: [N, C, H, W]
__device__ __host__ inline int idx4(int n, int c, int h, int w,
                                    int C, int H, int W) {
    return ((n * C + c) * H + h) * W + w;
}

// ==== KERNEL DECLARATIONS ====

__global__ void conv2d_forward_naive(
    const float* __restrict__ input,    // [N, C_in, H, W]
    const float* __restrict__ weight,   // [C_out, C_in, K, K]
    const float* __restrict__ bias,     // [C_out]
    float* __restrict__ output,         // [N, C_out, H_out, W_out]
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride);

__global__ void relu_forward(float* x, int size);

__global__ void maxpool2x2_forward(
    const float* __restrict__ input,  // [N, C, H, W]
    float* __restrict__ output,       // [N, C, H/2, W/2]
    int N, int C, int H, int W);

__global__ void upsample2x2_forward(
    const float* __restrict__ input,  // [N, C, H, W]
    float* __restrict__ output,       // [N, C, 2H, 2W]
    int N, int C, int H, int W);

__global__ void mse_loss_forward(
    const float* __restrict__ output,
    const float* __restrict__ target,
    float* __restrict__ loss,   // single float on device
    int size);

__global__ void relu_backward(
    const float* __restrict__ x,       // forward output/input to ReLU
    const float* __restrict__ grad_y,  // dL/dy
    float* __restrict__ grad_x,        // dL/dx
    int size);

__global__ void maxpool2x2_backward(
    const float* __restrict__ input,
    const float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W);

__global__ void upsample2x2_backward(
    const float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W);

__global__ void mse_loss_backward(
    const float* __restrict__ output,
    const float* __restrict__ target,
    float* __restrict__ grad_out,
    int size);

__global__ void conv2d_backward_input_naive(
    const float* __restrict__ dY,
    const float* __restrict__ weight,
    float* __restrict__ dX,
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride);

__global__ void conv2d_backward_weight_naive(
    const float* __restrict__ input,
    const float* __restrict__ dY,
    float* __restrict__ dW,
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride);

__global__ void conv2d_backward_bias_naive(
    const float* __restrict__ dY,
    float* __restrict__ dB,
    int N, int C_out, int H_out, int W_out);

__global__ void sgd_update(
    float* __restrict__ param,
    const float* __restrict__ grad,
    int size,
    float lr);


Overwriting gpu_layers.h


In [ ]:
%%writefile gpu_layers.cu
#include "gpu_layers.h"

// --------------- Conv2D forward (không đổi) ------------------
__global__ void conv2d_forward_naive(
    const float* __restrict__ input,
    const float* __restrict__ weight,
    const float* __restrict__ bias,
    float* __restrict__ output,
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc    = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n      = nc / C_out;
    int c_out  = nc % C_out;
    if (n >= N) return;

    float sum = bias ? bias[c_out] : 0.0f;

    for (int c_in = 0; c_in < C_in; ++c_in) {
        for (int kh = 0; kh < K; ++kh) {
            for (int kw = 0; kw < K; ++kw) {
                int h_in = h_out * stride + kh - pad;
                int w_in = w_out * stride + kw - pad;
                if (h_in < 0 || h_in >= H || w_in < 0 || w_in >= W)
                    continue;

                int in_idx = ((n * C_in + c_in) * H + h_in) * W + w_in;
                int w_idx = (((c_out * C_in + c_in) * K) + kh) * K + kw;
                sum += weight[w_idx] * input[in_idx];
            }
        }
    }
    int out_idx = ((n * C_out + c_out) * H_out + h_out) * W_out + w_out;
    output[out_idx] = sum;
}

// --------------- ReLU ------------------
__global__ void relu_forward(float* x, int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        float v = x[i];
        x[i] = v > 0.0f ? v : 0.0f;
    }
}

// --------------- MaxPool 2x2 ------------------
__global__ void maxpool2x2_forward(
    const float* __restrict__ input,
    float* __restrict__ output,
    int N, int C, int H, int W)
{
    int H_out = H / 2;
    int W_out = W / 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc    = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in0 = h_out * 2;
    int w_in0 = w_out * 2;

    float m = -1e30f;
    for (int dh = 0; dh < 2; ++dh) {
        for (int dw = 0; dw < 2; ++dw) {
            int h_in = h_in0 + dh;
            int w_in = w_in0 + dw;
            int idx = idx4(n, c, h_in, w_in, C, H, W);
            float v = input[idx];
            if (v > m) m = v;
        }
    }

    int out_idx = idx4(n, c, h_out, w_out, C, H_out, W_out);
    output[out_idx] = m;
}

// --------------- UpSample 2x2 ------------------
__global__ void upsample2x2_forward(
    const float* __restrict__ input,
    float* __restrict__ output,
    int N, int C, int H, int W)
{
    int H_out = H * 2;
    int W_out = W * 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc    = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in = h_out / 2;
    int w_in = w_out / 2;

    int idx_in  = idx4(n, c, h_in, w_in, C, H, W);
    int idx_out = idx4(n, c, h_out, w_out, C, H_out, W_out);
    output[idx_out] = input[idx_in];
}

// --------------- MSE loss ------------------
__global__ void mse_loss_forward(
    const float* __restrict__ output,
    const float* __restrict__ target,
    float* __restrict__ loss,
    int size)
{
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    float val = 0.0f;
    if (idx < size) {
        float diff = output[idx] - target[idx];
        val = diff * diff;
    }
    sdata[tid] = val;
    __syncthreads();

    // reduce trong block
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        atomicAdd(loss, sdata[0]);
    }
}


// --------------- ReLU backward ------------------
__global__ void relu_backward(
    const float* __restrict__ x,
    const float* __restrict__ grad_y,
    float* __restrict__ grad_x,
    int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        float v = x[i];
        grad_x[i] = (v > 0.0f) ? grad_y[i] : 0.0f;
    }
}

// --------------- MaxPool 2x2 backward - FIX ------------------
// BUG FIX: Chỉ ghi vào vị trí max, không ghi đè 4 vị trí
__global__ void maxpool2x2_backward(
    const float* __restrict__ input,
    const float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W)
{
    int H_out = H / 2;
    int W_out = W / 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc    = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in0 = h_out * 2;
    int w_in0 = w_out * 2;

    int idx00 = idx4(n, c, h_in0 + 0, w_in0 + 0, C, H, W);
    int idx01 = idx4(n, c, h_in0 + 0, w_in0 + 1, C, H, W);
    int idx10 = idx4(n, c, h_in0 + 1, w_in0 + 0, C, H, W);
    int idx11 = idx4(n, c, h_in0 + 1, w_in0 + 1, C, H, W);

    float v00 = input[idx00];
    float v01 = input[idx01];
    float v10 = input[idx10];
    float v11 = input[idx11];

    float g = grad_out[idx4(n, c, h_out, w_out, C, H_out, W_out)];

    // Tìm max
    float m = v00;
    int max_idx = 0;
    if (v01 > m) { m = v01; max_idx = 1; }
    if (v10 > m) { m = v10; max_idx = 2; }
    if (v11 > m) { m = v11; max_idx = 3; }

    // FIX: Chỉ ghi vào vị trí max (grad_in đã được zero trước đó)
    // Mỗi pooling window độc lập, không có conflict
    if (max_idx == 0) grad_in[idx00] = g;
    else if (max_idx == 1) grad_in[idx01] = g;
    else if (max_idx == 2) grad_in[idx10] = g;
    else grad_in[idx11] = g;
}

// --------------- UpSample 2x2 backward ------------------
__global__ void upsample2x2_backward(
    const float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W)
{
    int H_out = H * 2;
    int W_out = W * 2;

    int w_in = blockIdx.x * blockDim.x + threadIdx.x;
    int h_in = blockIdx.y * blockDim.y + threadIdx.y;
    int nc   = blockIdx.z;

    if (w_in >= W || h_in >= H) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_out0 = h_in * 2;
    int w_out0 = w_in * 2;

    float sum = 0.0f;
    for (int dh = 0; dh < 2; ++dh) {
        for (int dw = 0; dw < 2; ++dw) {
            int h_out = h_out0 + dh;
            int w_out = w_out0 + dw;
            int idx_o = idx4(n, c, h_out, w_out, C, H_out, W_out);
            sum += grad_out[idx_o];
        }
    }

    int idx_in = idx4(n, c, h_in, w_in, C, H, W);
    grad_in[idx_in] = sum;
}

// --------------- MSE loss backward ------------------
__global__ void mse_loss_backward(
    const float* __restrict__ output,
    const float* __restrict__ target,
    float* __restrict__ grad_out,
    int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= size) return;
    grad_out[i] = 2.0f * (output[i] - target[i]) / size;
}

// --------------- Conv2D backward: dX ------------------
__global__ void conv2d_backward_input_naive(
    const float* __restrict__ dY,
    const float* __restrict__ weight,
    float* __restrict__ dX,
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    int w = blockIdx.x * blockDim.x + threadIdx.x;
    int h = blockIdx.y * blockDim.y + threadIdx.y;
    int nc = blockIdx.z;

    if (w >= W || h >= H) return;

    int n = nc / C_in;
    int c_in = nc % C_in;
    if (n >= N) return;

    float sum = 0.0f;
    for (int c_out = 0; c_out < C_out; ++c_out) {
        for (int kh = 0; kh < K; ++kh) {
            for (int kw = 0; kw < K; ++kw) {
                int h_out = h + pad - kh;
                int w_out = w + pad - kw;

                if (h_out % stride != 0 || w_out % stride != 0) continue;

                h_out /= stride;
                w_out /= stride;

                if (h_out < 0 || h_out >= H_out ||
                    w_out < 0 || w_out >= W_out)
                    continue;

                int dy_idx = idx4(n, c_out, h_out, w_out,
                                  C_out, H_out, W_out);

                int kh_flip = K - 1 - kh;
                int kw_flip = K - 1 - kw;
                int w_idx = (((c_out * C_in + c_in) * K) + kh_flip) * K + kw_flip;
                sum += dY[dy_idx] * weight[w_idx];
            }
        }
    }

    int dx_idx = idx4(n, c_in, h, w, C_in, H, W);
    dX[dx_idx] = sum;
}

// --------------- Conv2D backward: dW ------------------
// Mỗi thread tính toàn bộ gradient cho 1 weight element
// Không có conflict vì mỗi thread ghi vào vị trí riêng biệt
__global__ void conv2d_backward_weight_naive(
    const float* __restrict__ input,
    const float* __restrict__ dY,
    float* __restrict__ dW,
    int N, int C_in, int H, int W,
    int C_out, int K, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = C_out * C_in * K * K;
    if (idx >= total) return;

    int kw = idx % K;
    int tmp = idx / K;
    int kh = tmp % K;
    tmp /= K;
    int c_in = tmp % C_in;
    int c_out = tmp / C_in;

    float sum = 0.0f;
    for (int n = 0; n < N; ++n) {
        for (int h_out = 0; h_out < H_out; ++h_out) {
            for (int w_out = 0; w_out < W_out; ++w_out) {
                int h_in = h_out * stride + kh - pad;
                int w_in = w_out * stride + kw - pad;
                if (h_in < 0 || h_in >= H || w_in < 0 || w_in >= W)
                    continue;

                int in_idx = idx4(n, c_in, h_in, w_in, C_in, H, W);
                int dy_idx = idx4(n, c_out, h_out, w_out,
                                  C_out, H_out, W_out);
                sum += dY[dy_idx] * input[in_idx];
            }
        }
    }

    // Mỗi thread ghi vào vị trí riêng, không conflict
    dW[idx] += sum;
}

// --------------- Conv2D backward: dB ------------------
// Mỗi thread tính gradient cho 1 bias element
__global__ void conv2d_backward_bias_naive(
    const float* __restrict__ dY,
    float* __restrict__ dB,
    int N, int C_out, int H_out, int W_out)
{
    int c_out = blockIdx.x * blockDim.x + threadIdx.x;
    if (c_out >= C_out) return;

    float sum = 0.0f;
    for (int n = 0; n < N; ++n) {
        for (int h = 0; h < H_out; ++h) {
            for (int w = 0; w < W_out; ++w) {
                int idx = idx4(n, c_out, h, w, C_out, H_out, W_out);
                sum += dY[idx];
            }
        }
    }

    // Mỗi thread ghi vào vị trí riêng, không conflict
    dB[c_out] += sum;
}

// --------------- SGD update ------------------
__global__ void sgd_update(
    float* __restrict__ param,
    const float* __restrict__ grad,
    int size,
    float lr)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        param[i] -= lr * grad[i];
    }
}

Overwriting gpu_layers.cu


In [ ]:
%%writefile cpu_layers.h
#pragma once
#include <stdio.h>
#include <float.h>

void Relu(float* input, int N, float* output);
void Conv2D_Forward(float* input, int input_width, int input_height, int input_channels,
    float* kernel, int kernel_width, int kernel_height,
    float* biases, int padding, int stride, int filter_count,
    float* output, int output_height, int output_width);
void MaxPool2D_Forward(float* input, int input_width, int input_height,
    int filter_width, int filter_height, int stride, int filter_count,
    float* output, int output_height, int output_width);
void UpSample2D_Forward(float* input, int input_width, int input_height,
    int scale_factor, int filter_count,
    float* output, int output_height, int output_width);
float MSE(float* input, float* output, int size);
void Relu_Backward(float* d_output, float* input,int N);
void MSE_Gradient(float* input, float* output, int size, float* d_output);
void UpSample2D_Backward(float* d_output, int d_output_width, int d_output_height, int scale_factor, int filter_count,
    float* d_input, int d_input_height, int d_input_width);
void MaxPool2D_Backward(float* d_output, int d_output_width, int d_output_height, float* input,
    int input_width, int input_height, int filter_width, int filter_height, int stride, int filter_count, float* d_input);
void Conv2D_Backward_Input(float* d_output, int d_output_width, int d_output_height, float* kernel, int kernel_width, int kernel_height,
    int input_width, int input_height, int input_channels, int padding, int stride, int filter_count, float* d_input);
void Conv2D_Backward_Kernel(float* d_output, int d_output_width, int d_output_height, float* input,
    int input_width, int input_height, int input_channels, int kernel_width, int kernel_height, int padding, int stride, int filter_count, float* d_weights);
void Conv2D_Backward_Biases(float* d_output, int d_output_width, int d_output_height, int filter_count, float* d_biases);
void SGD_Update(float* weights, float* d_weights, double learning_rate, int N_params);

Overwriting cpu_layers.h


In [ ]:
%%writefile cpu_layers.c
#include "cpu_layers.h"

void Relu(float* input, int N, float* output) {
    for (int i = 0; i < N; i++) {
        output[i] = input[i] > 0.0f ? input[i] : 0.0f;
    }
}

void Conv2D_Forward(float* input, int input_width, int input_height, int input_channels,
    float* kernel, int kernel_width, int kernel_height,
    float* biases, int padding, int stride, int filter_count,
    float* output, int output_height, int output_width)
{
    // // Tính toán kích thước output
    // int H_out = (input_height + 2 * padding - kernel_height) / stride + 1;
    // int W_out = (input_width + 2 * padding - kernel_width) / stride + 1;
    // int output_size = filter_count * H_out * W_out;
    // output = (float*)malloc(output_size * sizeof(float));
    // if (output == NULL) {
    //     fprintf(stderr, "ERROR: Memory allocation failed!\n");
    //     return;
    // }
    // Lặp qua kênh đầu ra (filter)
    for (int c_out = 0; c_out < filter_count; c_out++) {
        // Lặp qua chiều cao output
        for (int h_out = 0; h_out < output_height; h_out++) {
            // Lặp qua chiều rộng output
            for (int w_out = 0; w_out < output_width; w_out++) {
                float sum = 0.0f;
                // Lặp qua kênh đầu vào (c_in)
                for (int c_in = 0; c_in < input_channels; c_in++) {
                    // Lặp qua kernel height
                    for (int k_h = 0; k_h < kernel_height; k_h++) {
                        // Lặp qua kernel width
                        for (int k_w = 0; k_w < kernel_width; k_w++) {
                            // Vị trí input tương ứng
                            int h_in = h_out * stride + k_h - padding;
                            int w_in = w_out * stride + k_w - padding;
                            float val = 0.0f;
                            // Kiểm tra zero padding
                            if (h_in >= 0 && h_in < input_height && w_in >= 0 && w_in < input_width) {
                                int channel_size = input_width * input_height;
                                val = input[c_in * channel_size + h_in * input_width + w_in];
                            }

                            int weight_idx = c_out * input_channels * kernel_height * kernel_width + c_in * kernel_height * kernel_width +
                                            k_h * kernel_width + k_w;

                            sum += val * kernel[weight_idx];
                        }
                    }
                }
                sum += biases[c_out];  // Thêm bias
                int output_idx = h_out * output_width + w_out + c_out * output_width * output_height;
                output[output_idx] = sum;
            }
        }
    }
}


void MaxPool2D_Forward(float* input, int input_width, int input_height, int filter_width, int filter_height, int stride, int filter_count,
    float* output, int output_height, int output_width) {
    // int H_out = (input_height - filter_height) / stride + 1;
    // int W_out = (input_width - filter_width) / stride + 1;

    int plane_size_in = input_height * input_width;
    int plane_size_out = output_height * output_width;
    for (int c = 0; c < filter_count; c++) {
        for (int h_out = 0; h_out < output_height; h_out++) {
            for (int w_out = 0; w_out < output_width; w_out++) {
                float max_val = -FLT_MAX;
                int h_start = h_out * stride;
                int w_start = w_out * stride;
                for (int fh = 0; fh < filter_height; fh++) {
                    for (int fw = 0; fw < filter_width; fw++) {
                        int h_in = h_start + fh;
                        int w_in = w_start + fw;
                        int input_idx = c * plane_size_in + h_in * input_width + w_in;
                        float val = input[input_idx];
                        if (val > max_val) {
                            max_val = val;
                        }
                    }
                }
                int output_idx = c * plane_size_out + h_out * output_width + w_out;
                output[output_idx] = max_val;
            }
        }
    }
}

void UpSample2D_Forward(float* input, int input_width, int input_height,
int scale_factor, int filter_count, float* output, int output_height, int output_width) {
    int plane_size_in = input_height * input_width;
    int plane_size_out = output_height * output_width;
    for (int c = 0; c < filter_count; c++) {
        for (int h_in = 0; h_in < input_height; h_in++) {
            for (int w_in = 0; w_in < input_width; w_in++) {
                float val = input[c * plane_size_in + h_in * input_width + w_in];
                for (int sh = 0; sh < scale_factor; sh++) { // Gấp đôi hàng
                    for (int sw = 0; sw < scale_factor; sw++) { // Gấp đôi cột
                        int h_out = h_in * scale_factor + sh;
                        int w_out = w_in * scale_factor + sw;
                        int output_idx = c * plane_size_out + h_out * output_width + w_out;
                        output[output_idx] = val;
                    }
                }
            }
        }
    }
}

float MSE(float* input, float* output, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; i++) {
        sum += (output[i] - input[i]) * (output[i] - input[i]);
    }
    return sum / size;
}

void Relu_Backward(float* d_output, float* input,int N) {
    for (int i = 0; i < N; i++) {
        d_output[i] = input[i] > 0.0f ? d_output[i] : 0.0f;
    }
}

void MSE_Gradient(float* input, float* output, int size, float* d_output) {
    float sum = 0.0f;
    float factor = 2.0f / size;
    for (int i = 0; i < size; i++) {
        d_output[i] = factor * (output[i] - input[i]);
    }
}

void MaxPool2D_Backward(float* d_output, int d_output_width, int d_output_height, float* input,
    int input_width, int input_height, int filter_width, int filter_height, int stride, int filter_count,
    float* d_input)
{
    // Chỉ gán vị trí giá trị max của input ban đầu là gradient của lớp tiếp theo (d_output), còn lại là 0
    int plane_size_in = input_height * input_width;
    int plane_size_out = d_output_height * d_output_width;
    int total_input_size = filter_count * plane_size_in;
    for (int i = 0; i < total_input_size; i++) { // Khởi tạo gradient của input ban đầu là 0
        d_input[i] = 0.0f;
    }

    for (int c = 0; c < filter_count; c++) {

        int channel_offset_in = c * plane_size_in;
        int channel_offset_out = c * plane_size_out;

        for (int h_out = 0; h_out < d_output_height; h_out++) {
            for (int w_out = 0; w_out < d_output_width; w_out++) {

                int h_start = h_out * stride;
                int w_start = w_out * stride;

                float max_val = -FLT_MAX;
                int max_input_idx = -1;

                for (int fh = 0; fh < filter_height; fh++) {
                    for (int fw = 0; fw < filter_width; fw++) {
                        int h_in = h_start + fh;
                        int w_in = w_start + fw;
                        int input_idx = channel_offset_in + h_in * input_width + w_in;
                        float val = input[input_idx];
                        if (val > max_val) {
                            max_val = val;
                            max_input_idx = input_idx;
                        }
                    }
                }
                //Lấy gradient từ output
                if (max_input_idx != -1) {
                    int output_idx = channel_offset_out + h_out * d_output_width + w_out;
                    d_input[max_input_idx] += d_output[output_idx];
                }
            }
        }
    }
}

void UpSample2D_Backward(float* d_output, int d_output_width, int d_output_height, int scale_factor, int filter_count,
    float* d_input, int d_input_height, int d_input_width) {

    int plane_size_in = d_input_height * d_input_width;
    int plane_size_out = d_output_height * d_output_width;
    int total_input_size = filter_count * plane_size_in;
    for (int i = 0; i < total_input_size; i++) {
        d_input[i] = 0.0f;
    }
    for (int c = 0; c < filter_count; c++) {
        int channel_offset_in = c * plane_size_in;
        int channel_offset_out = c * plane_size_out;
        // Lặp qua input grid (d_input)
        for (int h_in = 0; h_in < d_input_height; h_in++) {
            for (int w_in = 0; w_in < d_input_width; w_in++) {
                float sum_gradient = 0.0f;
                int h_start_out = h_in * scale_factor;
                int w_start_out = w_in * scale_factor;
                for (int sh = 0; sh < scale_factor; sh++) {
                    for (int sw = 0; sw < scale_factor; sw++) {
                        int h_out = h_start_out + sh;
                        int w_out = w_start_out + sw;
                        if (h_out < d_output_height && w_out < d_output_width) {
                            int output_idx = channel_offset_out + h_out * d_output_width + w_out;
                            sum_gradient += d_output[output_idx];
                        }
                    }
                }
                int input_idx = channel_offset_in + h_in * d_input_width + w_in;
                d_input[input_idx] = sum_gradient;
            }
        }
    }
}

void Conv2D_Backward_Input(float* d_output, int d_output_width, int d_output_height, float* kernel, int kernel_width, int kernel_height,
    int input_width, int input_height, int input_channels, int padding, int stride, int filter_count, float* d_input) {
    // Thực hiện tích chập giữa dE/dO và kernel (xoay 180 độ) để tính dE/dI
    int plane_size_in = input_height * input_width;
    int plane_size_out = d_output_height * d_output_width;
    // Lặp qua kênh input (kênh output gradient)
    for (int c_in = 0; c_in < input_channels; c_in++) {
        // Lặp qua input grid (d_input)
        for (int h_in = 0; h_in < input_height; h_in++) {
            for (int w_in = 0; w_in < input_width; w_in++) {
                float sum_gradient = 0.0f;
                // Lặp qua kênh output (số lượng filter)
                for (int c_out = 0; c_out < filter_count; c_out++) {
                    // Lặp qua kernel (xoay 180 độ)
                    for (int kh = 0; kh < kernel_height; kh++) {
                        for (int kw = 0; kw < kernel_width; kw++) {
                            int h_out = h_in - kh + padding;
                            int w_out = w_in - kw + padding;
                            float d_output_val = 0.0f;
                            // Kiểm tra padding
                            if (h_out >= 0 && h_out < d_output_height && w_out >= 0 && w_out < d_output_width) {
                                int d_output_idx = c_out * plane_size_out + h_out * d_output_width + w_out;
                                d_output_val = d_output[d_output_idx];
                            }
                            // Tính chỉ số kernel (xoay 180 độ)
                            int kernel_idx = c_out * input_channels * kernel_height * kernel_width + c_in * kernel_height * kernel_width +
                                            (kernel_height - 1 - kh) * kernel_width + (kernel_width - 1 - kw);

                            sum_gradient += d_output_val * kernel[kernel_idx];
                        }
                    }
                }
                int d_input_idx = c_in * plane_size_in + h_in * input_width + w_in;
                d_input[d_input_idx] = sum_gradient;
            }
        }
    }
}

void Conv2D_Backward_Kernel(float* d_output, int d_output_width, int d_output_height, float* input,
int input_width, int input_height, int input_channels, int kernel_width, int kernel_height, int padding, int stride, int filter_count, float* d_weights) {
    // Thực hiện tích chập giữa dE/dO và input để tính dE/dW
    int plane_size_in = input_height * input_width;
    int plane_size_out = d_output_height * d_output_width;
    // Lặp qua kênh đầu ra (filter)
    for (int c_out = 0; c_out < filter_count; c_out++) {
        // Lặp qua kênh đầu vào
        for (int c_in = 0; c_in < input_channels; c_in++) {
            // Lặp qua kích thước kernel
            for (int k_h = 0; k_h < kernel_height; k_h++) {
                for (int k_w = 0; k_w < kernel_width; k_w++) {
                    float sum_gradient = 0.0f;
                    // Lặp qua output grid (d_output) để tích lũy
                    for (int h_out = 0; h_out < d_output_height; h_out++) {
                        for (int w_out = 0; w_out < d_output_width; w_out++) {
                            int h_in = h_out * stride + k_h - padding;
                            int w_in = w_out * stride + k_w - padding;
                            float input_val = 0.0f;
                            // Kiểm tra padding
                            if (h_in >= 0 && h_in < input_height && w_in >= 0 && w_in < input_width) {
                                int input_idx = c_in * plane_size_in + h_in * input_width + w_in;
                                input_val = input[input_idx];
                            }
                            int d_output_idx = c_out * plane_size_out + h_out * d_output_width + w_out;
                            float d_output_val = d_output[d_output_idx];
                            // Tính gradient
                            sum_gradient += input_val * d_output_val;
                        }
                    }
                    int kernel_idx = c_out * input_channels * kernel_height * kernel_width + c_in * kernel_height * kernel_width +
                                    k_h * kernel_width + k_w;
                    d_weights[kernel_idx] += sum_gradient;
                }
            }
        }
    }
}

void Conv2D_Backward_Biases(float* d_output, int d_output_width, int d_output_height,
    int filter_count, float* d_biases) {
    int plane_size_out = d_output_height * d_output_width;
    // Lặp qua từng filter
    for (int c_out = 0; c_out < filter_count; c_out++) {
        float sum_gradient = 0.0f;
        // Lặp qua từng vị trí trong output
        for (int h_out = 0; h_out < d_output_height; h_out++) {
            for (int w_out = 0; w_out < d_output_width; w_out++) {
                // Tính chỉ số trong mảng d_output
                int output_idx = c_out * plane_size_out + h_out * d_output_width + w_out;
                // Cộng dồn gradient từ d_output
                sum_gradient += d_output[output_idx];
            }
        }
        d_biases[c_out] += sum_gradient;
    }
}

void SGD_Update(float* weights, float* d_weights, double learning_rate, int N_params) {
    for (int i = 0; i < N_params; i++) {
        weights[i] -= (learning_rate * d_weights[i]);
    }
}

Overwriting cpu_layers.c


In [ ]:
%%writefile verify_gpu_cpu_layers.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

extern "C" {
    #include "cpu_layers.h"   // đã có sẵn
}

#include "gpu_layers.h"       // chứa các kernel GPU + CHECK_CUDA

// Hàm tiện ích so sánh 2 mảng
void compare_arrays(const char* name,
                    const float* a,
                    const float* b,
                    int n)
{
    double max_abs = 0.0;
    double sum_abs = 0.0;
    double sum_sq  = 0.0;

    for (int i = 0; i < n; ++i) {
        double diff = (double)a[i] - (double)b[i];
        double ad   = fabs(diff);

        if (ad > max_abs) max_abs = ad;
        sum_abs += ad;
        sum_sq  += diff * diff;
    }

    double mean_abs = sum_abs / n;
    double rmse     = std::sqrt(sum_sq / n);

    printf("[%s] max|diff| = %.6g, mean|diff| = %.6g, RMSE = %.6g\n",
           name, max_abs, mean_abs, rmse);
}

/*-------------------------------------------------------------
  TEST CONV2D (forward + backward) – cái này bạn đang chạy OK,
  mình để ví dụ đơn giản N=1 cho gọn.
-------------------------------------------------------------*/
void test_conv2d()
{
    printf("==================== test_conv2d ====================\n");

    int N      = 1;
    int C_in   = 3;
    int C_out  = 4;
    int H      = 8;
    int W      = 8;
    int K      = 3;
    int pad    = 1;
    int stride = 1;

    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    int in_size   = N * C_in * H * W;
    int w_size    = C_out * C_in * K * K;
    int b_size    = C_out;
    int out_size  = N * C_out * H_out * W_out;

    // Cấp phát host
    float *h_x     = (float*)malloc(in_size  * sizeof(float));
    float *h_w     = (float*)malloc(w_size   * sizeof(float));
    float *h_b     = (float*)malloc(b_size   * sizeof(float));
    float *h_y_cpu = (float*)malloc(out_size * sizeof(float));
    float *h_y_gpu = (float*)malloc(out_size * sizeof(float));

    float *h_dy    = (float*)malloc(out_size * sizeof(float));
    float *h_dx_cpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dx_gpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dw_cpu= (float*)malloc(w_size   * sizeof(float));
    float *h_dw_gpu= (float*)malloc(w_size   * sizeof(float));
    float *h_db_cpu= (float*)malloc(b_size   * sizeof(float));
    float *h_db_gpu= (float*)malloc(b_size   * sizeof(float));

    // Init dữ liệu determinisitc
    for (int i = 0; i < in_size; ++i)  h_x[i] = (float)((i % 17) - 8) / 10.0f;
    for (int i = 0; i < w_size; ++i)   h_w[i] = (float)((i % 13) - 6) / 7.0f;
    for (int i = 0; i < b_size; ++i)   h_b[i] = (float)((i % 5) - 2) / 3.0f;
    for (int i = 0; i < out_size; ++i) h_dy[i]= (float)((i % 11) - 5) / 9.0f;

    // ===== CPU FORWARD =====
    Conv2D_Forward(
        h_x, W, H, C_in,
        h_w, K, K,
        h_b,
        pad, stride,
        C_out,
        h_y_cpu,
        H_out, W_out);

    // ===== GPU FORWARD =====
    float *d_x=nullptr, *d_w=nullptr, *d_b=nullptr, *d_y=nullptr;
    CHECK_CUDA(cudaMalloc(&d_x, in_size  * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_w, w_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, b_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_y, out_size * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_x, h_x, in_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_w, h_w, w_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_b, h_b, b_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block2d(16,16);
    dim3 gridConv(
        (W_out + block2d.x - 1)/block2d.x,
        (H_out + block2d.y - 1)/block2d.y,
        N * C_out);

    conv2d_forward_naive<<<gridConv, block2d>>>(
        d_x, d_w, d_b, d_y,
        N, C_in, H, W,
        C_out, K, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_y_gpu, d_y, out_size * sizeof(float), cudaMemcpyDeviceToHost));

    compare_arrays("Conv2D_Forward", h_y_cpu, h_y_gpu, out_size);

    // ===== CPU BACKWARD =====
    Conv2D_Backward_Input(
        h_dy, W_out, H_out,
        h_w, K, K,
        W, H, C_in,
        pad, stride, C_out,
        h_dx_cpu);

    Conv2D_Backward_Kernel(
        h_dy, W_out, H_out,
        h_x, W, H, C_in,
        K, K,
        pad, stride, C_out,
        h_dw_cpu);

    Conv2D_Backward_Biases(
        h_dy, W_out, H_out,
        C_out,
        h_db_cpu);

    // ===== GPU BACKWARD =====
    float *d_dy=nullptr, *d_dx=nullptr, *d_dw=nullptr, *d_db=nullptr;
    CHECK_CUDA(cudaMalloc(&d_dy, out_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dx, in_size  * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dw, w_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_db, b_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_dy, h_dy, out_size * sizeof(float), cudaMemcpyHostToDevice));

    // dX
    dim3 gridIn(
        (W + block2d.x - 1)/block2d.x,
        (H + block2d.y - 1)/block2d.y,
        N * C_in);
    conv2d_backward_input_naive<<<gridIn, block2d>>>(
        d_dy, d_w, d_dx,
        N, C_in, H, W,
        C_out, K, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dW
    int num_w = w_size;
    int t = 256;
    int b = (num_w + t - 1)/t;
    conv2d_backward_weight_naive<<<b, t>>>(
        d_x, d_dy, d_dw,
        N, C_in, H, W,
        C_out, K, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dB
    int tb = 256;
    int bb = (C_out + tb - 1)/tb;
    conv2d_backward_bias_naive<<<bb, tb>>>(
        d_dy, d_db,
        N, C_out, H_out, W_out);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_dx_gpu, d_dx, in_size  * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_dw_gpu, d_dw, w_size   * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_db_gpu, d_db, b_size   * sizeof(float), cudaMemcpyDeviceToHost));

    compare_arrays("Conv2D_Backward_Input (dX)", h_dx_cpu, h_dx_gpu, in_size);
    compare_arrays("Conv2D_Backward_Kernel (dW)", h_dw_cpu, h_dw_gpu, w_size);
    compare_arrays("Conv2D_Backward_Biases (dB)", h_db_cpu, h_db_gpu, b_size);

    // cleanup
    free(h_x); free(h_w); free(h_b); free(h_y_cpu); free(h_y_gpu);
    free(h_dy); free(h_dx_cpu); free(h_dx_gpu);
    free(h_dw_cpu); free(h_dw_gpu);
    free(h_db_cpu); free(h_db_gpu);

    cudaFree(d_x); cudaFree(d_w); cudaFree(d_b); cudaFree(d_y);
    cudaFree(d_dy); cudaFree(d_dx); cudaFree(d_dw); cudaFree(d_db);
}

/*-------------------------------------------------------------
  TEST MaxPool + UpSample (forward + backward)
  Quan trọng: truyền đúng H,W (H_input/H_pool), không dùng H_out nhầm.
-------------------------------------------------------------*/
void test_pool_upsample()
{
    printf("\n==================== test_pool_upsample ====================\n");

    int N = 1;
    int C = 3;
    int H = 8;
    int W = 8;

    int pool_k     = 2;
    int pool_stride= 2;
    int H_pool     = H / pool_k;  // 4
    int W_pool     = W / pool_k;  // 4

    int scale      = 2;
    int H_up       = H_pool * scale; // 8
    int W_up       = W_pool * scale; // 8

    int in_size    = N * C * H      * W;
    int pool_size  = N * C * H_pool * W_pool;
    int up_size    = N * C * H_up   * W_up;

    // Host buffers
    float *h_in              = (float*)malloc(in_size   * sizeof(float));
    float *h_pool_cpu        = (float*)malloc(pool_size * sizeof(float));
    float *h_pool_gpu        = (float*)malloc(pool_size * sizeof(float));
    float *h_up_cpu          = (float*)malloc(up_size   * sizeof(float));
    float *h_up_gpu          = (float*)malloc(up_size   * sizeof(float));
    float *h_pool_grad       = (float*)malloc(pool_size * sizeof(float));
    float *h_in_grad_cpu     = (float*)malloc(in_size   * sizeof(float));
    float *h_in_grad_gpu     = (float*)malloc(in_size   * sizeof(float));
    float *h_up_grad         = (float*)malloc(up_size   * sizeof(float));
    float *h_pool_grad2_cpu  = (float*)malloc(pool_size * sizeof(float));
    float *h_pool_grad2_gpu  = (float*)malloc(pool_size * sizeof(float));

    // Init input
    for (int i = 0; i < in_size; ++i)
        h_in[i] = (float)((i % 13) - 6) / 7.0f;

    // ===== CPU MaxPool Forward =====
    MaxPool2D_Forward(
        h_in,
        W, H,
        pool_k, pool_k,
        pool_stride,
        C,
        h_pool_cpu,
        H_pool, W_pool);

    // ===== GPU MaxPool Forward =====
    float *d_in=nullptr, *d_pool=nullptr, *d_up=nullptr;
    float *d_pool_grad=nullptr, *d_in_grad=nullptr;
    float *d_up_grad=nullptr, *d_pool_grad2=nullptr;

    CHECK_CUDA(cudaMalloc(&d_in,   in_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_pool, pool_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_up,   up_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_in, h_in, in_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block2d(16,16);
    dim3 gridPool(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    // GPU pool fwd: H,W là kích thước input (8x8)
    maxpool2x2_forward<<<gridPool, block2d>>>(
        d_in, d_pool,
        N, C, H, W);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_pool_gpu, d_pool, pool_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("MaxPool2D_Forward", h_pool_cpu, h_pool_gpu, pool_size);

    // ===== CPU MaxPool Backward =====
    for (int i = 0; i < pool_size; ++i)
        h_pool_grad[i] = (float)((i % 7) - 3) / 5.0f;

    MaxPool2D_Backward(
        h_pool_grad, W_pool, H_pool,
        h_in,
        W, H,
        pool_k, pool_k, pool_stride,
        C,
        h_in_grad_cpu);

    // ===== GPU MaxPool Backward =====
    CHECK_CUDA(cudaMalloc(&d_pool_grad, pool_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_in_grad,   in_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_pool_grad, h_pool_grad, pool_size * sizeof(float),
                          cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_in_grad, 0, in_size * sizeof(float)));

    dim3 gridPoolB(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    maxpool2x2_backward<<<gridPoolB, block2d>>>(
        d_in, d_pool_grad, d_in_grad,
        N, C, H, W);   // H,W = kích thước INPUT (8x8)
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_in_grad_gpu, d_in_grad, in_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("MaxPool2D_Backward", h_in_grad_cpu, h_in_grad_gpu, in_size);

    // ===== CPU UpSample Forward (4x4 -> 8x8) =====
    UpSample2D_Forward(
        h_pool_cpu,
        W_pool, H_pool,
        scale,
        C,
        h_up_cpu,
        H_up, W_up);

    // ===== GPU UpSample Forward =====
    dim3 gridUp(
        (W_up + block2d.x - 1) / block2d.x,
        (H_up + block2d.y - 1) / block2d.y,
        N * C);

    upsample2x2_forward<<<gridUp, block2d>>>(
        d_pool, d_up,
        N, C,
        H_pool, W_pool);   // H,W = kích thước input (4x4)
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_up_gpu, d_up, up_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("UpSample2D_Forward", h_up_cpu, h_up_gpu, up_size);

    // ===== CPU UpSample Backward =====
    for (int i = 0; i < up_size; ++i)
        h_up_grad[i] = (float)((i % 11) - 5) / 9.0f;

    UpSample2D_Backward(
        h_up_grad,
        W_up, H_up,
        scale,
        C,
        h_pool_grad2_cpu,
        H_pool, W_pool);

    // ===== GPU UpSample Backward =====
    CHECK_CUDA(cudaMalloc(&d_up_grad,    up_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_pool_grad2, pool_size * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_up_grad, h_up_grad, up_size * sizeof(float),
                          cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_pool_grad2, 0, pool_size * sizeof(float)));

    dim3 gridUpB(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    upsample2x2_backward<<<gridUpB, block2d>>>(
        d_up_grad, d_pool_grad2,
        N, C,
        H_pool, W_pool);   // CHÚ Ý: H,W = kích thước INPUT (4x4), KHÔNG phải 8x8
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_pool_grad2_gpu, d_pool_grad2, pool_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("UpSample2D_Backward", h_pool_grad2_cpu, h_pool_grad2_gpu, pool_size);

    // cleanup
    free(h_in);
    free(h_pool_cpu); free(h_pool_gpu);
    free(h_up_cpu);   free(h_up_gpu);
    free(h_pool_grad);
    free(h_in_grad_cpu); free(h_in_grad_gpu);
    free(h_up_grad);
    free(h_pool_grad2_cpu); free(h_pool_grad2_gpu);

    cudaFree(d_in);
    cudaFree(d_pool);
    cudaFree(d_up);
    cudaFree(d_pool_grad);
    cudaFree(d_in_grad);
    cudaFree(d_up_grad);
    cudaFree(d_pool_grad2);
}

int main()
{
    test_conv2d();
    test_pool_upsample();

    CHECK_CUDA(cudaDeviceReset());
    return 0;
}


Overwriting verify_gpu_cpu_layers.cu


In [ ]:
!nvcc -arch=sm_75 -O2 \
  verify_gpu_cpu_layers.cu gpu_layers.cu cpu_layers.c \
  -o verify_kernels

In [ ]:
!./verify_kernels

==================== test_conv2d ====================
[Conv2D_Forward] max|diff| = 5.96046e-07, mean|diff| = 1.04599e-07, RMSE = 1.44526e-07
[Conv2D_Backward_Input (dX)] max|diff| = 2.38419e-07, mean|diff| = 5.21541e-08, RMSE = 8.97555e-08
[Conv2D_Backward_Kernel (dW)] max|diff| = 4.76837e-07, mean|diff| = 6.1864e-08, RMSE = 1.04859e-07
[Conv2D_Backward_Biases (dB)] max|diff| = 0, mean|diff| = 0, RMSE = 0

==================== test_pool_upsample ====================
[MaxPool2D_Forward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[MaxPool2D_Backward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[UpSample2D_Forward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[UpSample2D_Backward] max|diff| = 0, mean|diff| = 0, RMSE = 0


## GPU Optimization Version 1:

In [ ]:
%%writefile gpu_layers_opt1.h
#pragma once
#include <cuda_runtime.h>
#include <stdio.h>

#define TILE_W 16
#define TILE_H 16
// Kích thước Kernel
#define K 3
#define R (K/2) // Radius = 1
#define BLOCK_W (TILE_W + 2 * R)
#define BLOCK_H (TILE_H + 2 * R)
__constant__ float dc_bias[256];


#define CHECK_CUDA(call)                                                \
    do {                                                                \
        cudaError_t err = call;                                         \
        if (err != cudaSuccess) {                                       \
            fprintf(stderr, "CUDA Error %s:%d: %s\n",                   \
                    __FILE__, __LINE__, cudaGetErrorString(err));       \
            exit(EXIT_FAILURE);                                         \
        }                                                               \
    } while (0)

// NCHW layout: [N, C, H, W]
__device__ __host__ inline int idx4(int n, int c, int h, int w,
                                    int C, int H, int W) {
    return ((n * C + c) * H + h) * W + w;
}

// ==== KERNEL DECLARATIONS ====
void update_dc_bias(float* d_bias_ptr, int count);
__global__ void conv2d_forward(
    float* __restrict__ input,    // [N, C_in, H, W]
    float* __restrict__ weight,   // [C_out, C_in, K, K]
    // float* bias,     // [C_out]
    float* __restrict__ output,   // [N, C_out, H_out, W_out]
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride);

__global__ void relu_forward(float* x, int size);

__global__ void maxpool2x2_forward(
    float* __restrict__ input,  // [N, C, H, W]
    float* __restrict__ output,       // [N, C, H/2, W/2]
    int N, int C, int H, int W);

__global__ void upsample2x2_forward(
    float* __restrict__ input,  // [N, C, H, W]
    float* __restrict__ output,       // [N, C, 2H, 2W]
    int N, int C, int H, int W);

__global__ void mse_loss_forward(
    float* __restrict__ output,
    float* __restrict__ target,
    float* __restrict__ loss,   // single float on device
    int size);

__global__ void relu_backward(
    float* __restrict__ x,       // forward output/input to ReLU
    float* __restrict__ grad_y,  // dL/dy
    float* __restrict__ grad_x,        // dL/dx
    int size);

__global__ void maxpool2x2_backward(
    float* __restrict__ input,
    float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W);

__global__ void upsample2x2_backward(
    float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W);


__global__ void mse_loss_backward(
    float* __restrict__ output,
    float* __restrict__ target,
    float* __restrict__ grad_out,
    int size);

__global__ void conv2d_backward_input(
    float* __restrict__ dY,
    float* __restrict__ weight,
    float* __restrict__ dX,
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride);

__global__ void conv2d_backward_weight(
    float* __restrict__ input,
    float* __restrict__ dY,
    float* __restrict__ dW,
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride);

__global__ void conv2d_backward_bias(
    float* __restrict__ dY,
    float* __restrict__ dB,
    int N, int C_out, int H_out, int W_out);

__global__ void sgd_update(
    float* __restrict__ param,
    float* __restrict__ grad,
    int size,
    float lr);

Overwriting gpu_layers_opt1.h


In [ ]:
%%writefile gpu_layers_opt1.cu
#include "gpu_layers_opt1.h"

// --------------- Conv2D forward (optimization 1) ------------------
void update_dc_bias(float* d_bias_ptr, int count) {
    CHECK_CUDA(cudaMemcpyToSymbol(dc_bias, d_bias_ptr, count * sizeof(float),
                                   0, cudaMemcpyDeviceToDevice));
}

__global__ void conv2d_forward(
    float* __restrict__ input,    // [N, C_in, H, W]
    float* __restrict__ weight,   // [C_out, C_in, K, K]
    float* __restrict__ output,   // [N, C_out, H_out, W_out]
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    __shared__ float smem[BLOCK_H][BLOCK_W];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int w_out = blockIdx.x * TILE_W + tx;
    int h_out = blockIdx.y * TILE_H + ty;

    int nc = blockIdx.z;
    int c_out = nc % C_out;
    int n = nc / C_out;

    float value = 0.0f;

    int row_start = blockIdx.y * TILE_H * stride - pad;
    int col_start = blockIdx.x * TILE_W * stride - pad;

    for (int c_in = 0; c_in < C_in; c_in++) {
        // Load input tile into shared memory
        for (int i = ty; i < BLOCK_H; i += blockDim.y) {
            for (int j = tx; j < BLOCK_W; j += blockDim.x) {
                int h_in = row_start + i;
                int w_in = col_start + j;

                if (h_in >= 0 && h_in < H && w_in >= 0 && w_in < W) {
                    size_t input_idx = idx4(n, c_in, h_in, w_in, C_in, H, W);
                    smem[i][j] = input[input_idx];
                } else {
                    smem[i][j] = 0.0f;
                }
            }
        }
        __syncthreads();

        // Compute convolution
        for (int i = 0; i < K; ++i) {
            for (int j = 0; j < K; ++j) {
                size_t weight_idx = idx4(c_out, c_in, i, j, C_in, K, K);
                value += smem[ty + i][tx + j] * weight[weight_idx];
            }
        }
        __syncthreads();
    }

    if (w_out < W_out && h_out < H_out && n < N) {
        value += dc_bias[c_out];
        output[idx4(n, c_out, h_out, w_out, C_out, H_out, W_out)] = value;
    }
}

// --------------- ReLU ------------------
__global__ void relu_forward(float* __restrict__ x, int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        float v = x[i];
        x[i] = (v > 0.0f) ? v : 0.0f;
    }
}

// --------------- MaxPool 2x2 (stride 2) ------------------
__global__ void maxpool2x2_forward(
    float* __restrict__ input,
    float* __restrict__ output,
    int N, int C, int H, int W)
{
    int H_out = H / 2;
    int W_out = W / 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in0 = h_out * 2;
    int w_in0 = w_out * 2;

    float m = -1e30f;
    for (int dh = 0; dh < 2; dh++) {
        for (int dw = 0; dw < 2; dw++) {
            int h_in = h_in0 + dh;
            int w_in = w_in0 + dw;
            int idx = idx4(n, c, h_in, w_in, C, H, W);
            float v = input[idx];
            if (v > m) m = v;
        }
    }

    int out_idx = idx4(n, c, h_out, w_out, C, H_out, W_out);
    output[out_idx] = m;
}

// --------------- UpSample 2x2 (nearest) ------------------
__global__ void upsample2x2_forward(
    float* __restrict__ input,
    float* __restrict__ output,
    int N, int C, int H, int W)
{
    int H_out = H * 2;
    int W_out = W * 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in = h_out / 2;
    int w_in = w_out / 2;

    int idx_in = idx4(n, c, h_in, w_in, C, H, W);
    int idx_out = idx4(n, c, h_out, w_out, C, H_out, W_out);
    output[idx_out] = input[idx_in];
}

// --------------- MSE loss ------------------
__global__ void mse_loss_forward(
    float* __restrict__ output,
    float* __restrict__ target,
    float* __restrict__ loss,
    int size)
{
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    float val = 0.0f;
    if (idx < size) {
        float diff = output[idx] - target[idx];
        val = diff * diff;
    }
    sdata[tid] = val;
    __syncthreads();

    // Parallel reduction
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        atomicAdd(loss, sdata[0]);
    }
}

// --------------- ReLU backward ------------------
__global__ void relu_backward(
    float* __restrict__ x,
    float* __restrict__ grad_y,
    float* __restrict__ grad_x,
    int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        grad_x[i] = (x[i] > 0.0f) ? grad_y[i] : 0.0f;
    }
}

// --------------- MaxPool 2x2 backward ------------------
__global__ void maxpool2x2_backward(
    float* __restrict__ input,
    float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W)
{
    int H_out = H / 2;
    int W_out = W / 2;

    int w_out = blockIdx.x * blockDim.x + threadIdx.x;
    int h_out = blockIdx.y * blockDim.y + threadIdx.y;
    int nc = blockIdx.z;

    if (w_out >= W_out || h_out >= H_out) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_in0 = h_out * 2;
    int w_in0 = w_out * 2;

    int idx00 = idx4(n, c, h_in0 + 0, w_in0 + 0, C, H, W);
    int idx01 = idx4(n, c, h_in0 + 0, w_in0 + 1, C, H, W);
    int idx10 = idx4(n, c, h_in0 + 1, w_in0 + 0, C, H, W);
    int idx11 = idx4(n, c, h_in0 + 1, w_in0 + 1, C, H, W);

    float v00 = input[idx00];
    float v01 = input[idx01];
    float v10 = input[idx10];
    float v11 = input[idx11];

    float g = grad_out[idx4(n, c, h_out, w_out, C, H_out, W_out)];

    float m = v00;
    int max_idx = 0;
    if (v01 > m) { m = v01; max_idx = 1; }
    if (v10 > m) { m = v10; max_idx = 2; }
    if (v11 > m) { m = v11; max_idx = 3; }

    if (max_idx == 0) grad_in[idx00] = g;
    else if (max_idx == 1) grad_in[idx01] = g;
    else if (max_idx == 2) grad_in[idx10] = g;
    else grad_in[idx11] = g;
}

// --------------- UpSample 2x2 backward ------------------
__global__ void upsample2x2_backward(
    float* __restrict__ grad_out,
    float* __restrict__ grad_in,
    int N, int C, int H, int W)
{
    int H_out = H * 2;
    int W_out = W * 2;

    int w_in = blockIdx.x * blockDim.x + threadIdx.x;
    int h_in = blockIdx.y * blockDim.y + threadIdx.y;
    int nc = blockIdx.z;

    if (w_in >= W || h_in >= H) return;

    int n = nc / C;
    int c = nc % C;
    if (n >= N) return;

    int h_out0 = h_in * 2;
    int w_out0 = w_in * 2;

    float sum = 0.0f;
    for (int dh = 0; dh < 2; ++dh) {
        for (int dw = 0; dw < 2; ++dw) {
            int h_out = h_out0 + dh;
            int w_out = w_out0 + dw;
            int idx_o = idx4(n, c, h_out, w_out, C, H_out, W_out);
            sum += grad_out[idx_o];
        }
    }

    int idx_in = idx4(n, c, h_in, w_in, C, H, W);
    grad_in[idx_in] = sum;
}

// --------------- MSE loss backward ------------------
__global__ void mse_loss_backward(
    float* __restrict__ output,
    float* __restrict__ target,
    float* __restrict__ grad_out,
    int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= size) return;
    grad_out[i] = 2.0f * (output[i] - target[i]) / size;
}

// --------------- Conv2D backward: dX ------------------
__global__ void conv2d_backward_input(
    float* __restrict__ dY,
    float* __restrict__ weight,
    float* __restrict__ dX,
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    // Shared Memory chứa dY
    __shared__ float s_dY[BLOCK_H][BLOCK_W];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Tọa độ dX (output của kernel này)
    int h_out = blockIdx.y * TILE_H + ty;
    int w_out = blockIdx.x * TILE_W + tx;

    int nc = blockIdx.z;
    int c_in = nc % C_in;
    int n = nc / C_in;
    //if (h_out >= H || w_out >= W || n >= N) return;

    float value = 0.0f;

    for (int c_out = 0; c_out < C_out; ++c_out) {
        // Tọa độ dY cần tải
        int h_global = h_out;
        int w_global = w_out;

        // Tải main tile
        if (h_global >= 0 && h_global < H_out && w_global >= 0 && w_global < W_out) {
            s_dY[ty + pad][tx + pad] = dY[idx4(n, c_out, h_global, w_global, C_out, H_out, W_out)];
        } else {
            s_dY[ty + pad][tx + pad] = 0.0f;
        }

        // Tải biên trên/dưới
        if (ty < pad) {
            // Biên trên
            int h_top = blockIdx.y * TILE_H + ty - pad;
            if (h_top >= 0 && h_top < H_out && w_global >= 0 && w_global < W_out) {
                s_dY[ty][tx + pad] = dY[idx4(n, c_out, h_top, w_global, C_out, H_out, W_out)];
            } else {
                s_dY[ty][tx + pad] = 0.0f;
            }

            // Biên dưới
            int h_bottom = blockIdx.y * TILE_H + TILE_H + pad - 1 - ty;
            if (h_bottom >= 0 && h_bottom < H_out && w_global >= 0 && w_global < W_out) {
                s_dY[BLOCK_H - 1 - ty][tx + pad] = dY[idx4(n, c_out, h_bottom, w_global, C_out, H_out, W_out)];
            } else {
                s_dY[BLOCK_H - 1 - ty][tx + pad] = 0.0f;
            }
        }

        // Tải biên trái/phải
        if (tx < pad) {
            // Biên trái
            int w_left = blockIdx.x * TILE_W + tx - pad;
            if (w_left >= 0 && w_left < W_out && h_global >= 0 && h_global < H_out) {
                s_dY[ty + pad][tx] = dY[idx4(n, c_out, h_global, w_left, C_out, H_out, W_out)];
            } else {
                s_dY[ty + pad][tx] = 0.0f;
            }

            // Biên phải
            int w_right = blockIdx.x * TILE_W + TILE_W + pad - 1 - tx;
            if (w_right >= 0 && w_right < W_out && h_global >= 0 && h_global < H_out) {
                s_dY[ty + pad][BLOCK_W - 1 - tx] = dY[idx4(n, c_out, h_global, w_right, C_out, H_out, W_out)];
            } else {
                s_dY[ty + pad][BLOCK_W - 1 - tx] = 0.0f;
            }
        }

        // Tải 4 góc (Thread (0,0) tải)
        if (tx == 0 && ty == 0) {
            // Góc trên trái [0][0]
            int h_c = blockIdx.y * TILE_H - pad;
            int w_c = blockIdx.x * TILE_W - pad;
            s_dY[0][0] = (h_c >= 0 && h_c < H_out && w_c >= 0 && w_c < W_out) ? dY[idx4(n, c_out, h_c, w_c, C_out, H_out, W_out)] : 0.0f;

            // Góc trên phải [0][17]
            w_c = blockIdx.x * TILE_W + TILE_W + pad - 1;
            s_dY[0][BLOCK_W - 1] = (h_c >= 0 && h_c < H_out && w_c >= 0 && w_c < W_out) ? dY[idx4(n, c_out, h_c, w_c, C_out, H_out, W_out)] : 0.0f;

            // Góc dưới trái [17][0]
            h_c = blockIdx.y * TILE_H + TILE_H + pad - 1;
            w_c = blockIdx.x * TILE_W - pad;
            s_dY[BLOCK_H - 1][0] = (h_c >= 0 && h_c < H_out && w_c >= 0 && w_c < W_out) ? dY[idx4(n, c_out, h_c, w_c, C_out, H_out, W_out)] : 0.0f;

            // Góc dưới phải [17][17]
            w_c = blockIdx.x * TILE_W + TILE_W + pad - 1;
            s_dY[BLOCK_H - 1][BLOCK_W - 1] = (h_c >= 0 && h_c < H_out && w_c >= 0 && w_c < W_out) ? dY[idx4(n, c_out, h_c, w_c, C_out, H_out, W_out)] : 0.0f;
        }

        __syncthreads();

        // Tính convolution
            for (int kh = 0; kh < K; ++kh) {
                for (int kw = 0; kw < K; ++kw) {
                    int smem_y = ty + 2 * pad - kh;
                    int smem_x = tx + 2* pad - kw;

                    size_t w_idx = idx4(c_out, c_in, K - 1 - kh, K - 1 - kw, C_in, K, K);
                    value += s_dY[smem_y][smem_x] * weight[w_idx];
                }
            }
        __syncthreads();
    }

    if (h_out < H && w_out < W && n < N) {
        dX[idx4(n, c_in, h_out, w_out, C_in, H, W)] = value;
    }
}

// --------------- Conv2D backward: dW ------------------
__global__ void conv2d_backward_weight(
    float* __restrict__ input,
    float* __restrict__ dY,
    float* __restrict__ dW,
    int N, int C_in, int H, int W,
    int C_out, int pad, int stride)
{
    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    __shared__ float s_in[BLOCK_H][BLOCK_W];
    __shared__ float s_dY[TILE_H][TILE_W];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int index = blockIdx.z;
    int c_in = index % C_in;
    int c_out = index / C_in;

    float dw[K][K];
    for (int i = 0; i < K; ++i)
        for (int j = 0; j < K; ++j)
            dw[i][j] = 0.0f;

    for (int n = 0; n < N; ++n) {
        int num_blocks_h = (H_out + TILE_H - 1) / TILE_H;
        int num_blocks_w = (W_out + TILE_W - 1) / TILE_W;

        for (int block_h = 0; block_h < num_blocks_h; ++block_h) {
            for (int block_w = 0; block_w < num_blocks_w; ++block_w) {

                // Load s_dY
                int h_out = block_h * TILE_H + ty;
                int w_out = block_w * TILE_W + tx;

                if (h_out < H_out && w_out < W_out) {
                    s_dY[ty][tx] = dY[idx4(n, c_out, h_out, w_out, C_out, H_out, W_out)];
                } else {
                    s_dY[ty][tx] = 0.0f;
                }

                // Load s_in
                int h_in_base = block_h * TILE_H + ty;
                int w_in_base = block_w * TILE_W + tx;

                // Main tile
                if (h_in_base >= 0 && h_in_base < H && w_in_base >= 0 && w_in_base < W) {
                    s_in[ty + pad][tx + pad] = input[idx4(n, c_in, h_in_base, w_in_base, C_in, H, W)];
                } else {
                    s_in[ty + pad][tx + pad] = 0.0f;
                }

                // Top/bottom borders
                if (ty < pad) {
                    int h_top = block_h * TILE_H - pad + ty;
                    if (h_top >= 0 && h_top < H && w_in_base >= 0 && w_in_base < W) {
                        s_in[ty][tx + pad] = input[idx4(n, c_in, h_top, w_in_base, C_in, H, W)];
                    } else {
                        s_in[ty][tx + pad] = 0.0f;
                    }

                    int h_bottom = block_h * TILE_H + TILE_H + pad - 1 - ty;
                    if (h_bottom >= 0 && h_bottom < H && w_in_base >= 0 && w_in_base < W) {
                        s_in[BLOCK_H - 1 - ty][tx + pad] = input[idx4(n, c_in, h_bottom, w_in_base, C_in, H, W)];
                    } else {
                        s_in[BLOCK_H - 1 - ty][tx + pad] = 0.0f;
                    }
                }

                // Left/right borders
                if (tx < pad) {
                    int w_left = block_w * TILE_W - pad + tx;
                    if (w_left >= 0 && w_left < W && h_in_base >= 0 && h_in_base < H) {
                        s_in[ty + pad][tx] = input[idx4(n, c_in, h_in_base, w_left, C_in, H, W)];
                    } else {
                        s_in[ty + pad][tx] = 0.0f;
                    }

                    int w_right = block_w * TILE_W + TILE_W + pad - 1 - tx;
                    if (w_right >= 0 && w_right < W && h_in_base >= 0 && h_in_base < H) {
                        s_in[ty + pad][BLOCK_W - 1 - tx] = input[idx4(n, c_in, h_in_base, w_right, C_in, H, W)];
                    } else {
                        s_in[ty + pad][BLOCK_W - 1 - tx] = 0.0f;
                    }
                }

                // Thread (0,0) loads 4 corners
                if (tx == 0 && ty == 0) {
                    int h_c = block_h * TILE_H - pad;
                    int w_c = block_w * TILE_W - pad;
                    s_in[0][0] = (h_c >= 0 && h_c < H && w_c >= 0 && w_c < W)
                        ? input[idx4(n, c_in, h_c, w_c, C_in, H, W)] : 0.0f;

                    w_c = block_w * TILE_W + TILE_W + pad - 1;
                    s_in[0][BLOCK_W - 1] = (h_c >= 0 && h_c < H && w_c >= 0 && w_c < W)
                        ? input[idx4(n, c_in, h_c, w_c, C_in, H, W)] : 0.0f;

                    h_c = block_h * TILE_H + TILE_H + pad - 1;
                    w_c = block_w * TILE_W - pad;
                    s_in[BLOCK_H - 1][0] = (h_c >= 0 && h_c < H && w_c >= 0 && w_c < W)
                        ? input[idx4(n, c_in, h_c, w_c, C_in, H, W)] : 0.0f;

                    w_c = block_w * TILE_W + TILE_W + pad - 1;
                    s_in[BLOCK_H - 1][BLOCK_W - 1] = (h_c >= 0 && h_c < H && w_c >= 0 && w_c < W)
                        ? input[idx4(n, c_in, h_c, w_c, C_in, H, W)] : 0.0f;
                }

                __syncthreads();

                // Compute dW: X * dY
                float val_dy = s_dY[ty][tx];
                for (int kh = 0; kh < K; ++kh) {
                    for (int kw = 0; kw < K; ++kw) {
                        dw[kh][kw] += s_in[ty + kh][tx + kw] * val_dy;
                    }
                }

                __syncthreads();
            }
        }
    }

    // Write accumulated gradients
    for (int i = 0; i < K; ++i) {
        for (int j = 0; j < K; ++j) {
            // Tính index global của dW[i][j] cho cặp filter (c_out, c_in)
            size_t dw_idx = idx4(c_out, c_in, i, j, C_in, K, K);
            // Cộng giá trị dw[i][j] của thread hiện tại vào bộ nhớ global
            atomicAdd(&dW[dw_idx], dw[i][j]);
        }
    }
}

// --------------- Conv2D backward: dB ------------------
__global__ void conv2d_backward_bias(
    float* __restrict__ dY,
    float* __restrict__ dB,
    int N, int C_out, int H_out, int W_out)
{
    int spatial_size = H_out * W_out;
    int channel_size = N * spatial_size;
    extern __shared__ float sdata[];
    int tid = threadIdx.x;
    int c = blockIdx.x;
    if (c >= C_out) return;
    float sum = 0.0f;
    for (int i = tid; i < channel_size; i += blockDim.x) {
        int n = i / spatial_size;
        int rem = i % spatial_size;
        int global_idx = n * (C_out * spatial_size) + c * spatial_size + rem;
        sum += dY[global_idx];
    }
    sdata[tid] = sum;
    __syncthreads();

    // Reduction
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        dB[c] = sdata[0];
    }
}

// --------------- SGD update ------------------
__global__ void sgd_update(
    float* __restrict__ param,
    float* __restrict__ grad,
    int size,
    float lr)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        param[i] -= lr * grad[i];
    }
}

Overwriting gpu_layers_opt1.cu


In [ ]:
%%writefile verify_gpuOpt1_cpu_layers.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

extern "C" {
    #include "cpu_layers.h"   // đã có sẵn
}

#include "gpu_layers_opt1.h"       // chứa các kernel GPU + CHECK_CUDA

// Hàm tiện ích so sánh 2 mảng
void compare_arrays(const char* name,
                    const float* a,
                    const float* b,
                    int n)
{
    double max_abs = 0.0;
    double sum_abs = 0.0;
    double sum_sq  = 0.0;

    for (int i = 0; i < n; ++i) {
        double diff = (double)a[i] - (double)b[i];
        double ad   = fabs(diff);

        if (ad > max_abs) max_abs = ad;
        sum_abs += ad;
        sum_sq  += diff * diff;
    }

    double mean_abs = sum_abs / n;
    double rmse     = std::sqrt(sum_sq / n);

    printf("[%s] max|diff| = %.6g, mean|diff| = %.6g, RMSE = %.6g\n",
           name, max_abs, mean_abs, rmse);
}

/*-------------------------------------------------------------
  TEST CONV2D (forward + backward)
-------------------------------------------------------------*/
void test_conv2d()
{
    printf("==================== test_conv2d ====================\n");

    int N      = 1;
    int C_in   = 3;
    int C_out  = 4;
    int H      = 8;
    int W      = 8;
    //int K      = 3;
    int pad    = 1;
    int stride = 1;

    int H_out = (H + 2 * pad - K) / stride + 1;
    int W_out = (W + 2 * pad - K) / stride + 1;

    int in_size   = N * C_in * H * W;
    int w_size    = C_out * C_in * K * K;
    int b_size    = C_out;
    int out_size  = N * C_out * H_out * W_out;

    // Cấp phát host
    float *h_x     = (float*)malloc(in_size  * sizeof(float));
    float *h_w     = (float*)malloc(w_size   * sizeof(float));
    float *h_b     = (float*)malloc(b_size   * sizeof(float));
    float *h_y_cpu = (float*)malloc(out_size * sizeof(float));
    float *h_y_gpu = (float*)malloc(out_size * sizeof(float));

    float *h_dy    = (float*)malloc(out_size * sizeof(float));
    float *h_dx_cpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dx_gpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dw_cpu= (float*)malloc(w_size   * sizeof(float));
    float *h_dw_gpu= (float*)malloc(w_size   * sizeof(float));
    float *h_db_cpu= (float*)malloc(b_size   * sizeof(float));
    float *h_db_gpu= (float*)malloc(b_size   * sizeof(float));

    // Init dữ liệu determinisitc
    for (int i = 0; i < in_size; ++i)  h_x[i] = (float)((i % 17) - 8) / 10.0f;
    for (int i = 0; i < w_size; ++i)   h_w[i] = (float)((i % 13) - 6) / 7.0f;
    for (int i = 0; i < b_size; ++i)   h_b[i] = (float)((i % 5) - 2) / 3.0f;
    for (int i = 0; i < out_size; ++i) h_dy[i]= (float)((i % 11) - 5) / 9.0f;

    // ===== CPU FORWARD =====
    Conv2D_Forward(
        h_x, W, H, C_in,
        h_w, K, K,
        h_b,
        pad, stride,
        C_out,
        h_y_cpu,
        H_out, W_out);

    // ===== GPU FORWARD =====
    float *d_x=nullptr, *d_w=nullptr, *d_b=nullptr, *d_y=nullptr;
    CHECK_CUDA(cudaMalloc(&d_x, in_size  * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_w, w_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, b_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_y, out_size * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_x, h_x, in_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_w, h_w, w_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_b, h_b, b_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block2d(16,16);
    dim3 gridConv(
        (W_out + block2d.x - 1)/block2d.x,
        (H_out + block2d.y - 1)/block2d.y,
        N * C_out);
    update_dc_bias(d_b, C_out);

    conv2d_forward<<<gridConv, block2d>>>(
        d_x, d_w, d_y,
        N, C_in, H, W,
        C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_y_gpu, d_y, out_size * sizeof(float), cudaMemcpyDeviceToHost));

    compare_arrays("Conv2D_Forward", h_y_cpu, h_y_gpu, out_size);

    // ===== CPU BACKWARD =====
    Conv2D_Backward_Input(
        h_dy, W_out, H_out,
        h_w, K, K,
        W, H, C_in,
        pad, stride, C_out,
        h_dx_cpu);

    Conv2D_Backward_Kernel(
        h_dy, W_out, H_out,
        h_x, W, H, C_in,
        K, K,
        pad, stride, C_out,
        h_dw_cpu);

    Conv2D_Backward_Biases(
        h_dy, W_out, H_out,
        C_out,
        h_db_cpu);

    // ===== GPU BACKWARD =====
    float *d_dy=nullptr, *d_dx=nullptr, *d_dw=nullptr, *d_db=nullptr;
    CHECK_CUDA(cudaMalloc(&d_dy, out_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dx, in_size  * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dw, w_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_db, b_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_dy, h_dy, out_size * sizeof(float), cudaMemcpyHostToDevice));

    // dX
    dim3 gridIn(
        (W + block2d.x - 1)/block2d.x,
        (H + block2d.y - 1)/block2d.y,
        N * C_in);
    conv2d_backward_input<<<gridIn, block2d>>>(
        d_dy, d_w, d_dx,
        N, C_in, H, W,
        C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dW
    dim3 blockW(16, 16);
    dim3 gridW(1, 1, C_out * C_in);
    conv2d_backward_weight<<<gridW, blockW>>>(
        d_x, d_dy, d_dw,
        N, C_in, H, W,
        C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dB
    dim3 blockB(256);
    dim3 gridB(C_out);
    size_t shmem_size = 256 * sizeof(float);
    conv2d_backward_bias<<<gridB, blockB, shmem_size>>>(
        d_dy, d_db,
        N, C_out, H_out, W_out);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_dx_gpu, d_dx, in_size  * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_dw_gpu, d_dw, w_size   * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_db_gpu, d_db, b_size   * sizeof(float), cudaMemcpyDeviceToHost));

    compare_arrays("Conv2D_Backward_Input (dX)", h_dx_cpu, h_dx_gpu, in_size);
    compare_arrays("Conv2D_Backward_Kernel (dW)", h_dw_cpu, h_dw_gpu, w_size);
    compare_arrays("Conv2D_Backward_Biases (dB)", h_db_cpu, h_db_gpu, b_size);

    // cleanup
    free(h_x); free(h_w); free(h_b); free(h_y_cpu); free(h_y_gpu);
    free(h_dy); free(h_dx_cpu); free(h_dx_gpu);
    free(h_dw_cpu); free(h_dw_gpu);
    free(h_db_cpu); free(h_db_gpu);

    cudaFree(d_x); cudaFree(d_w); cudaFree(d_b); cudaFree(d_y);
    cudaFree(d_dy); cudaFree(d_dx); cudaFree(d_dw); cudaFree(d_db);
}

/*-------------------------------------------------------------
  TEST MaxPool + UpSample (forward + backward)
  Quan trọng: truyền đúng H,W (H_input/H_pool), không dùng H_out nhầm.
-------------------------------------------------------------*/
void test_pool_upsample()
{
    printf("\n==================== test_pool_upsample ====================\n");

    int N = 1;
    int C = 3;
    int H = 8;
    int W = 8;

    int pool_k     = 2;
    int pool_stride= 2;
    int H_pool     = H / pool_k;  // 4
    int W_pool     = W / pool_k;  // 4

    int scale      = 2;
    int H_up       = H_pool * scale; // 8
    int W_up       = W_pool * scale; // 8

    int in_size    = N * C * H      * W;
    int pool_size  = N * C * H_pool * W_pool;
    int up_size    = N * C * H_up   * W_up;

    // Host buffers
    float *h_in              = (float*)malloc(in_size   * sizeof(float));
    float *h_pool_cpu        = (float*)malloc(pool_size * sizeof(float));
    float *h_pool_gpu        = (float*)malloc(pool_size * sizeof(float));
    float *h_up_cpu          = (float*)malloc(up_size   * sizeof(float));
    float *h_up_gpu          = (float*)malloc(up_size   * sizeof(float));
    float *h_pool_grad       = (float*)malloc(pool_size * sizeof(float));
    float *h_in_grad_cpu     = (float*)malloc(in_size   * sizeof(float));
    float *h_in_grad_gpu     = (float*)malloc(in_size   * sizeof(float));
    float *h_up_grad         = (float*)malloc(up_size   * sizeof(float));
    float *h_pool_grad2_cpu  = (float*)malloc(pool_size * sizeof(float));
    float *h_pool_grad2_gpu  = (float*)malloc(pool_size * sizeof(float));

    // Init input
    for (int i = 0; i < in_size; ++i)
        h_in[i] = (float)((i % 13) - 6) / 7.0f;

    // ===== CPU MaxPool Forward =====
    MaxPool2D_Forward(
        h_in,
        W, H,
        pool_k, pool_k,
        pool_stride,
        C,
        h_pool_cpu,
        H_pool, W_pool);

    // ===== GPU MaxPool Forward =====
    float *d_in=nullptr, *d_pool=nullptr, *d_up=nullptr;
    float *d_pool_grad=nullptr, *d_in_grad=nullptr;
    float *d_up_grad=nullptr, *d_pool_grad2=nullptr;

    CHECK_CUDA(cudaMalloc(&d_in,   in_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_pool, pool_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_up,   up_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_in, h_in, in_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block2d(16,16);
    dim3 gridPool(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    // GPU pool fwd: H,W là kích thước input (8x8)
    maxpool2x2_forward<<<gridPool, block2d>>>(
        d_in, d_pool,
        N, C, H, W);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_pool_gpu, d_pool, pool_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("MaxPool2D_Forward", h_pool_cpu, h_pool_gpu, pool_size);

    // ===== CPU MaxPool Backward =====
    for (int i = 0; i < pool_size; ++i)
        h_pool_grad[i] = (float)((i % 7) - 3) / 5.0f;

    MaxPool2D_Backward(
        h_pool_grad, W_pool, H_pool,
        h_in,
        W, H,
        pool_k, pool_k, pool_stride,
        C,
        h_in_grad_cpu);

    // ===== GPU MaxPool Backward =====
    CHECK_CUDA(cudaMalloc(&d_pool_grad, pool_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_in_grad,   in_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_pool_grad, h_pool_grad, pool_size * sizeof(float),
                          cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_in_grad, 0, in_size * sizeof(float)));

    dim3 gridPoolB(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    maxpool2x2_backward<<<gridPoolB, block2d>>>(
        d_in, d_pool_grad, d_in_grad,
        N, C, H, W);   // H,W = kích thước INPUT (8x8)
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_in_grad_gpu, d_in_grad, in_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("MaxPool2D_Backward", h_in_grad_cpu, h_in_grad_gpu, in_size);

    // ===== CPU UpSample Forward (4x4 -> 8x8) =====
    UpSample2D_Forward(
        h_pool_cpu,
        W_pool, H_pool,
        scale,
        C,
        h_up_cpu,
        H_up, W_up);

    // ===== GPU UpSample Forward =====
    dim3 gridUp(
        (W_up + block2d.x - 1) / block2d.x,
        (H_up + block2d.y - 1) / block2d.y,
        N * C);

    upsample2x2_forward<<<gridUp, block2d>>>(
        d_pool, d_up,
        N, C,
        H_pool, W_pool);   // H,W = kích thước input (4x4)
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_up_gpu, d_up, up_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("UpSample2D_Forward", h_up_cpu, h_up_gpu, up_size);

    // ===== CPU UpSample Backward =====
    for (int i = 0; i < up_size; ++i)
        h_up_grad[i] = (float)((i % 11) - 5) / 9.0f;

    UpSample2D_Backward(
        h_up_grad,
        W_up, H_up,
        scale,
        C,
        h_pool_grad2_cpu,
        H_pool, W_pool);

    // ===== GPU UpSample Backward =====
    CHECK_CUDA(cudaMalloc(&d_up_grad,    up_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_pool_grad2, pool_size * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_up_grad, h_up_grad, up_size * sizeof(float),
                          cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_pool_grad2, 0, pool_size * sizeof(float)));

    dim3 gridUpB(
        (W_pool + block2d.x - 1) / block2d.x,
        (H_pool + block2d.y - 1) / block2d.y,
        N * C);

    upsample2x2_backward<<<gridUpB, block2d>>>(
        d_up_grad, d_pool_grad2,
        N, C,
        H_pool, W_pool);   // CHÚ Ý: H,W = kích thước INPUT (4x4), KHÔNG phải 8x8
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_pool_grad2_gpu, d_pool_grad2, pool_size * sizeof(float),
                          cudaMemcpyDeviceToHost));

    compare_arrays("UpSample2D_Backward", h_pool_grad2_cpu, h_pool_grad2_gpu, pool_size);

    // cleanup
    free(h_in);
    free(h_pool_cpu); free(h_pool_gpu);
    free(h_up_cpu);   free(h_up_gpu);
    free(h_pool_grad);
    free(h_in_grad_cpu); free(h_in_grad_gpu);
    free(h_up_grad);
    free(h_pool_grad2_cpu); free(h_pool_grad2_gpu);

    cudaFree(d_in);
    cudaFree(d_pool);
    cudaFree(d_up);
    cudaFree(d_pool_grad);
    cudaFree(d_in_grad);
    cudaFree(d_up_grad);
    cudaFree(d_pool_grad2);
}

int main()
{
    test_conv2d();
    test_pool_upsample();

    CHECK_CUDA(cudaDeviceReset());
    return 0;
}


Overwriting verify_gpuOpt1_cpu_layers.cu


In [ ]:
!nvcc -arch=sm_75 -O2 \
  verify_gpuOpt1_cpu_layers.cu gpu_layers_opt1.cu cpu_layers.c \
  -o verify_kernels

In [ ]:
!./verify_kernels

==================== test_conv2d ====================
[Conv2D_Forward] max|diff| = 2.38419e-07, mean|diff| = 6.0659e-08, RMSE = 1.02412e-07
[Conv2D_Backward_Input (dX)] max|diff| = 2.38419e-07, mean|diff| = 5.21541e-08, RMSE = 8.97555e-08
[Conv2D_Backward_Kernel (dW)] max|diff| = 5.96046e-07, mean|diff| = 1.45562e-07, RMSE = 1.94332e-07
[Conv2D_Backward_Biases (dB)] max|diff| = 1.19209e-07, mean|diff| = 4.09782e-08, RMSE = 6.18892e-08

==================== test_pool_upsample ====================
[MaxPool2D_Forward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[MaxPool2D_Backward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[UpSample2D_Forward] max|diff| = 0, mean|diff| = 0, RMSE = 0
[UpSample2D_Backward] max|diff| = 0, mean|diff| = 0, RMSE = 0


In [ ]:
%%writefile verify_gpuOpt1_cpu_layers.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cstring>
#include <cuda_runtime.h>

extern "C" {
    #include "cpu_layers.h"
}

#include "gpu_layers_opt1.h"

// Helper: In mảng
void print_array(const char* name, const float* arr, int n) {
    printf("%s: ", name);
    for (int i = 0; i < n; ++i) {
        printf("%.4f ", arr[i]);
    }
    printf("\n");
}

// Helper: So sánh
void compare_arrays(const char* name, const float* a, const float* b, int n) {
    double max_abs = 0.0;
    double sum_sq  = 0.0;
    for (int i = 0; i < n; ++i) {
        double diff = (double)a[i] - (double)b[i];
        double ad   = fabs(diff);
        if (ad > max_abs) max_abs = ad;
        sum_sq  += diff * diff;
    }
    printf("[%s] max|diff| = %.6g, RMSE = %.6g\n", name, max_abs, std::sqrt(sum_sq / n));
}

void test_conv2d()
{
    printf("==================== test_conv2d (PYTORCH DATA) ====================\n");

    // Config khớp với dữ liệu bạn đưa:
    // Input: [1, 3, 2, 2] -> N=1, C_in=3, H=2, W=2
    // Weight: [1, 3, 3, 3] -> C_out=1, K=3
    int N = 1;
    int C_in = 3;
    int C_out = 1;
    int H = 2;
    int W = 2;
    int pad = 1;
    int stride = 1;

    int H_out = (H + 2 * pad - K) / stride + 1; // (2+2-3)/1 + 1 = 2
    int W_out = (W + 2 * pad - K) / stride + 1; // 2

    int in_size   = N * C_in * H * W;
    int w_size    = C_out * C_in * K * K;
    int b_size    = C_out;
    int out_size  = N * C_out * H_out * W_out;

    // --- Alloc ---
    float *h_x     = (float*)malloc(in_size  * sizeof(float));
    float *h_w     = (float*)malloc(w_size   * sizeof(float));
    float *h_b     = (float*)malloc(b_size   * sizeof(float));
    float *h_dy    = (float*)malloc(out_size * sizeof(float));

    float *h_y_cpu = (float*)malloc(out_size * sizeof(float));
    float *h_y_gpu = (float*)malloc(out_size * sizeof(float));
    float *h_dx_cpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dx_gpu= (float*)malloc(in_size  * sizeof(float));
    float *h_dw_cpu= (float*)malloc(w_size   * sizeof(float));
    float *h_dw_gpu= (float*)malloc(w_size   * sizeof(float));
    float *h_db_cpu= (float*)malloc(b_size   * sizeof(float));
    float *h_db_gpu= (float*)malloc(b_size   * sizeof(float));

    // --- Init Data (SỬA LỖI CÚ PHÁP Ở ĐÂY) ---
    printf("\n--- INPUT DATA ---\n");

    // Input Tensor [1, 3, 2, 2]
    float raw_x[] = {
         0.5716f,  0.5095f, -0.4530f, -0.6242f, // Channel 0
         0.6942f, -1.3731f, -0.5511f, -1.0356f, // Channel 1
         0.4376f, -0.2553f, -0.3711f,  1.1056f  // Channel 2
    };
    memcpy(h_x, raw_x, sizeof(raw_x));
    print_array("h_x (Input)", h_x, in_size);

    // Weight Tensor [1, 3, 3, 3]
    float raw_w[] = {
        // Channel 0 (3x3)
        -1.9601f,  1.7431f,  1.9581f,
         0.8421f,  1.3120f, -0.4370f,
        -0.0117f,  0.5837f,  0.3514f,
        // Channel 1 (3x3)
        -0.8800f,  0.2600f, -0.7063f,
        -0.3977f,  1.5940f, -1.0829f,
        -0.0248f, -1.3013f,  1.1109f,
        // Channel 2 (3x3)
         0.7878f, -0.0066f, -0.5166f,
        -1.7315f,  1.4600f, -0.3321f,
        -1.0723f,  0.5769f,  1.2464f
    };
    memcpy(h_w, raw_w, sizeof(raw_w));
    print_array("h_w (Weights)", h_w, w_size);

    // Bias [1]
    float raw_b[] = {-1.9176f};
    memcpy(h_b, raw_b, sizeof(raw_b));
    print_array("h_b (Bias)", h_b, b_size);

    // dY (Grad Output) - Giả lập
    for (int i = 0; i < out_size; ++i) h_dy[i]= (float)((i % 11) - 5) / 9.0f;
    print_array("h_dy (Grad Output)", h_dy, out_size);

    // ================= CPU FORWARD =================
    Conv2D_Forward(h_x, W, H, C_in, h_w, K, K, h_b, pad, stride, C_out, h_y_cpu, H_out, W_out);

    // ================= GPU FORWARD =================
    float *d_x, *d_w, *d_b, *d_y;
    CHECK_CUDA(cudaMalloc(&d_x, in_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_w, w_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, b_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_y, out_size * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_x, h_x, in_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_w, h_w, w_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_b, h_b, b_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 block2d(16,16);
    dim3 gridConv(
        (W_out + block2d.x - 1)/block2d.x,
        (H_out + block2d.y - 1)/block2d.y,
        N * C_out);

    // FIX: Dùng hàm helper update_dc_bias đã khai báo
    update_dc_bias(d_b, C_out);

    conv2d_forward<<<gridConv, block2d>>>(d_x, d_w, d_y, N, C_in, H, W, C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());
    CHECK_CUDA(cudaMemcpy(h_y_gpu, d_y, out_size * sizeof(float), cudaMemcpyDeviceToHost));

    printf("\n--- FORWARD RESULTS ---\n");
    print_array("h_y_cpu", h_y_cpu, out_size);
    print_array("h_y_gpu", h_y_gpu, out_size);
    compare_arrays("Conv2D_Forward", h_y_cpu, h_y_gpu, out_size);

    // ================= CPU BACKWARD =================
    Conv2D_Backward_Input(h_dy, W_out, H_out, h_w, K, K, W, H, C_in, pad, stride, C_out, h_dx_cpu);

    memset(h_dw_cpu, 0, w_size * sizeof(float));
    Conv2D_Backward_Kernel(h_dy, W_out, H_out, h_x, W, H, C_in, K, K, pad, stride, C_out, h_dw_cpu);

    memset(h_db_cpu, 0, b_size * sizeof(float));
    Conv2D_Backward_Biases(h_dy, W_out, H_out, C_out, h_db_cpu);

    // ================= GPU BACKWARD =================
    float *d_dy, *d_dx, *d_dw, *d_db;
    CHECK_CUDA(cudaMalloc(&d_dy, out_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dx, in_size  * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dw, w_size   * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_db, b_size   * sizeof(float)));

    CHECK_CUDA(cudaMemcpy(d_dy, h_dy, out_size * sizeof(float), cudaMemcpyHostToDevice));

    // FIX: Memset GPU buffers về 0
    CHECK_CUDA(cudaMemset(d_dx, 0, in_size * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_dw, 0, w_size * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_db, 0, b_size * sizeof(float)));

    // dX
    dim3 gridIn((W + block2d.x - 1)/block2d.x, (H + block2d.y - 1)/block2d.y, N * C_in);
    conv2d_backward_input<<<gridIn, block2d>>>(d_dy, d_w, d_dx, N, C_in, H, W, C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dW
    dim3 blockW(16, 16);
    dim3 gridW(1, 1, C_out * C_in);
    conv2d_backward_weight<<<gridW, blockW>>>(d_x, d_dy, d_dw, N, C_in, H, W, C_out, pad, stride);
    CHECK_CUDA(cudaDeviceSynchronize());

    // dB
    dim3 blockB(256);
    dim3 gridB(C_out);
    conv2d_backward_bias<<<gridB, blockB, 256*sizeof(float)>>>(d_dy, d_db, N, C_out, H_out, W_out);
    CHECK_CUDA(cudaDeviceSynchronize());

    // Readback
    CHECK_CUDA(cudaMemcpy(h_dx_gpu, d_dx, in_size  * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_dw_gpu, d_dw, w_size   * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_db_gpu, d_db, b_size   * sizeof(float), cudaMemcpyDeviceToHost));

    printf("\n--- BACKWARD RESULTS (dX) ---\n");
    print_array("h_dx_cpu", h_dx_cpu, in_size);
    print_array("h_dx_gpu", h_dx_gpu, in_size);
    compare_arrays("Conv2D_Backward_Input (dX)", h_dx_cpu, h_dx_gpu, in_size);

    printf("\n--- BACKWARD RESULTS (dW) ---\n");
    print_array("h_dw_cpu", h_dw_cpu, w_size);
    print_array("h_dw_gpu", h_dw_gpu, w_size);
    compare_arrays("Conv2D_Backward_Kernel (dW)", h_dw_cpu, h_dw_gpu, w_size);

    printf("\n--- BACKWARD RESULTS (dB) ---\n");
    print_array("h_db_cpu", h_db_cpu, b_size);
    print_array("h_db_gpu", h_db_gpu, b_size);
    compare_arrays("Conv2D_Backward_Biases (dB)", h_db_cpu, h_db_gpu, b_size);

    // Free
    free(h_x); free(h_w); free(h_b); free(h_y_cpu); free(h_y_gpu);
    free(h_dy); free(h_dx_cpu); free(h_dx_gpu); free(h_dw_cpu); free(h_dw_gpu); free(h_db_cpu); free(h_db_gpu);
    cudaFree(d_x); cudaFree(d_w); cudaFree(d_b); cudaFree(d_y); cudaFree(d_dy); cudaFree(d_dx); cudaFree(d_dw); cudaFree(d_db);
}

void test_pool_upsample() {
    printf("[SKIP] test_pool_upsample (Already Passed)\n");
}

int main() {
    test_conv2d();
    // test_pool_upsample();
    CHECK_CUDA(cudaDeviceReset());
    return 0;
}

Overwriting verify_gpuOpt1_cpu_layers.cu
